In [27]:
import pandas as pd
import os

In [92]:
import os
import pyarrow as pa
import pyarrow.csv as pc_csv
import pyarrow.compute as pc_compute
import pyarrow.parquet as pq

def dict_encode_all_str_columns(table: pa.Table) -> pa.Table:
    """
    Dictionary encodes all string columns in the given PyArrow table.

    Args:
        table (pa.Table): The input PyArrow table.

    Returns:
        pa.Table: A new PyArrow table with string columns dictionary encoded.
    """
    new_columns = []
    for field in table.schema:
        column = table[field.name]
        if pa.types.is_string(field.type):
            encoded_column = pc_compute.dictionary_encode(column)
            new_columns.append(encoded_column)
            print(f"   Encoded column '{field.name}' as dictionary.")
        else:
            new_columns.append(column)
    return pa.Table.from_arrays(new_columns, names=table.column_names)

def to_parquet(project_path: str) -> None:
    """
    Converts CSV files for each day of the week into dictionary-encoded Parquet files.

    Args:
        project_path (str): The base path of the project containing the data.
    """
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
    data_dir = os.path.join(project_path, 'data', 'cicids2017')

    for day in days:
        csv_file = os.path.join(data_dir, f'{day}.csv')
        parquet_file = os.path.join(data_dir, f'{day}.parquet')
        print(f"Loading data for {day}: {csv_file}")

        try:
            # Read CSV into a PyArrow Table
            table = pc_csv.read_csv(csv_file)
            print(f"   Successfully read {csv_file} into PyArrow Table.")

            # Dictionary encode all string columns
            encoded_table = dict_encode_all_str_columns(table)

            # Write the encoded table to a Parquet file
            pq.write_table(encoded_table, parquet_file)
            print(f"   Successfully transformed {csv_file} to {parquet_file}\n")

        except FileNotFoundError:
            print(f"   File not found: {csv_file}. Skipping {day}.\n")
        except Exception as e:
            print(f"   Error transforming {csv_file} to Parquet: {e}. Skipping {day}.\n")

# Example usage:
# to_parquet('/path/to/your/project')


In [93]:
def print_file_size(path: str) -> None:
    for file in os.listdir(path):
        print(f"   {file}: {(os.path.getsize(os.path.join(path, file)) / 1_048_576):.0f} MB")

In [94]:
project_path = '/home/tan/Code/finalproject/xgboost-cicids2017'

# Step 1: Load Data
df = to_parquet(project_path)

Loading data for monday: /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/monday.csv
   Successfully read /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/monday.csv into PyArrow Table.
   Encoded column 'Flow ID' as dictionary.
   Encoded column 'Src IP' as dictionary.
   Encoded column 'Dst IP' as dictionary.
   Encoded column 'Label' as dictionary.
   Successfully transformed /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/monday.csv to /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/monday.parquet

Loading data for tuesday: /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/tuesday.csv
   Successfully read /home/tan/Code/finalproject/xgboost-cicids2017/data/cicids2017/tuesday.csv into PyArrow Table.
   Encoded column 'Flow ID' as dictionary.
   Encoded column 'Src IP' as dictionary.
   Encoded column 'Dst IP' as dictionary.
   Encoded column 'Label' as dictionary.
   Successfully transformed /home/tan/Code/finalp

In [95]:
print_file_size(os.path.join(project_path, 'data', 'cicids2017'))

   friday.csv: 272 MB
   wednesday.csv: 278 MB
   thursday.parquet: 62 MB
   tuesday.csv: 170 MB
   monday.csv: 198 MB
   wednesday.parquet: 88 MB
   monday.parquet: 71 MB
   tuesday.parquet: 62 MB
   friday.parquet: 89 MB
   thursday.csv: 181 MB


In [96]:
import pyarrow.parquet as pq

In [97]:
friday_table = pq.read_table(os.path.join(project_path, 'data', 'cicids2017', 'friday.parquet'))
print(friday_table.column_names)

['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd RST Flags', 'Bwd RST Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Cou

In [98]:
friday_table = friday_table.to_pandas()
friday_table.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   id                          547557 non-null  int64         
 1   Flow ID                     547557 non-null  category      
 2   Src IP                      547557 non-null  category      
 3   Src Port                    547557 non-null  int64         
 4   Dst IP                      547557 non-null  category      
 5   Dst Port                    547557 non-null  int64         
 6   Protocol                    547557 non-null  int64         
 7   Timestamp                   547557 non-null  datetime64[ns]
 8   Flow Duration               547557 non-null  int64         
 9   Total Fwd Packet            547557 non-null  int64         
 10  Total Bwd packets           547557 non-null  int64         
 11  Total Length of Fwd Packet  547557 non-

In [17]:
fdf

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,...,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,...,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,BENIGN,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,...,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,BENIGN,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,...,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,BENIGN,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,...,1968172,6.497443e+07,0.000000e+00,64974431,64974431,-1,-1,0,BENIGN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547552,547553,172.16.0.1-192.168.10.50-64318-2222-6,172.16.0.1,64318,192.168.10.50,2222,6,2017-07-07 18:09:11.499555,64,1,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,64,Portscan,-1
547553,547554,172.16.0.1-192.168.10.50-33248-9040-6,172.16.0.1,33248,192.168.10.50,9040,6,2017-07-07 17:54:38.857717,76,1,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,76,Portscan,-1
547554,547555,172.16.0.1-192.168.10.50-44033-1272-6,172.16.0.1,44033,192.168.10.50,1272,6,2017-07-07 17:52:03.285040,81,1,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,81,Portscan,-1
547555,547556,192.168.10.16-192.168.10.3-64748-53-17,192.168.10.16,64748,192.168.10.3,53,17,2017-07-07 18:33:45.701233,297,2,...,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,BENIGN,-1


In [18]:
fdf.describe()

,id,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Attempted Category
count,547557.000000,547557.000000,547557.000000,547557.000000,5.475570e+05,547557.000000,547557.000000,547557.000000,5.475570e+05,547557.000000,...,5.475570e+05,5.475570e+05,5.475570e+05,5.475570e+05,5.475570e+05,5.475570e+05,547557.000000,547557.000000,5.475570e+05,547557.000000
mean,273779.000000,47158.825030,2826.089161,9.785144,9.252255e+06,8.726116,9.444566,374.740299,1.408719e+04,129.866494,...,2.023936e+05,1.125818e+05,3.610678e+06,1.455170e+05,3.716373e+06,3.451524e+06,-0.999262,-0.998444,1.561664e+07,-0.985145
std,158066.235011,16039.868513,8604.258903,5.233530,2.609339e+07,648.644128,874.427110,3088.263160,1.958688e+06,495.923430,...,1.071617e+06,6.682971e+05,1.149280e+07,1.870863e+06,1.182962e+07,1.131402e+07,0.050051,0.093615,1.899453e+08,0.171725
min,1.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-1.000000,0.000000e+00,-1.000000
25%,136890.000000,39606.000000,53.000000,6.000000,7.100000e+01,1.000000,1.000000,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-1.000000,0.000000e+00,-1.000000
50%,273779.000000,52332.000000,80.000000,6.000000,3.114100e+04,2.000000,2.000000,41.000000,1.680000e+02,27.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-1.000000,4.800000e+01,-1.000000
75%,410668.000000,59207.000000,1055.000000,17.000000,5.032396e+06,8.000000,5.000000,88.000000,4.158000e+03,47.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-1.000000,4.507917e+06,-1.000000
max,547557.000000,65535.000000,65525.000000,17.000000,1.200000e+08,207963.000000,284603.000000,624776.000000,6.270395e+08,24820.000000,...,1.100975e+08,1.100975e+08,1.199912e+08,7.663521e+07,1.199912e+08,1.199912e+08,10.000000,8.000000,2.676050e+10,1.000000


In [19]:
fdf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          547557 non-null  int64  
 1   Flow ID                     547557 non-null  object 
 2   Src IP                      547557 non-null  object 
 3   Src Port                    547557 non-null  int64  
 4   Dst IP                      547557 non-null  object 
 5   Dst Port                    547557 non-null  int64  
 6   Protocol                    547557 non-null  int64  
 7   Timestamp                   547557 non-null  object 
 8   Flow Duration               547557 non-null  int64  
 9   Total Fwd Packet            547557 non-null  int64  
 10  Total Bwd packets           547557 non-null  int64  
 11  Total Length of Fwd Packet  547557 non-null  int64  
 12  Total Length of Bwd Packet  547557 non-null  int64  
 13  Fwd Packet Len

In [99]:
ff = pd.read_parquet(
    path=os.path.join(project_path, 'data', 'cicids2017', 'friday.parquet'),
)

In [100]:
ff.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   id                          547557 non-null  int64         
 1   Flow ID                     547557 non-null  category      
 2   Src IP                      547557 non-null  category      
 3   Src Port                    547557 non-null  int64         
 4   Dst IP                      547557 non-null  category      
 5   Dst Port                    547557 non-null  int64         
 6   Protocol                    547557 non-null  int64         
 7   Timestamp                   547557 non-null  datetime64[ns]
 8   Flow Duration               547557 non-null  int64         
 9   Total Fwd Packet            547557 non-null  int64         
 10  Total Bwd packets           547557 non-null  int64         
 11  Total Length of Fwd Packet  547557 non-

In [50]:
%timeit ff.groupby('Label').size()

14.5 ms ± 162 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%timeit f.groupby('Label').size()

25.7 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
f = pd.read_csv(os.path.join(project_path, 'data', 'cicids2017', 'friday.csv'), engine='pyarrow', da)
f.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          547557 non-null  int64  
 1   Flow ID                     547557 non-null  object 
 2   Src IP                      547557 non-null  object 
 3   Src Port                    547557 non-null  int64  
 4   Dst IP                      547557 non-null  object 
 5   Dst Port                    547557 non-null  int64  
 6   Protocol                    547557 non-null  int64  
 7   Timestamp                   547557 non-null  object 
 8   Flow Duration               547557 non-null  int64  
 9   Total Fwd Packet            547557 non-null  int64  
 10  Total Bwd packets           547557 non-null  int64  
 11  Total Length of Fwd Packet  547557 non-null  int64  
 12  Total Length of Bwd Packet  547557 non-null  int64  
 13  Fwd Packet Len

In [53]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype                 
---  ------                      --------------   -----                 
 0   id                          547557 non-null  int64[pyarrow]        
 1   Flow ID                     547557 non-null  string                
 2   Src IP                      547557 non-null  string                
 3   Src Port                    547557 non-null  int64[pyarrow]        
 4   Dst IP                      547557 non-null  string                
 5   Dst Port                    547557 non-null  int64[pyarrow]        
 6   Protocol                    547557 non-null  int64[pyarrow]        
 7   Timestamp                   547557 non-null  timestamp[ns][pyarrow]
 8   Flow Duration               547557 non-null  int64[pyarrow]        
 9   Total Fwd Packet            547557 non-null  int64[pyarrow]        
 10  Total Bw

In [55]:
ff['Label'].value_counts()

Label
BENIGN                288544
Portscan              159066
DDoS                   95144
Botnet - Attempted      4067
Botnet                   736
Name: count, dtype: int64[pyarrow]

In [56]:
ff['Label'] = ff['Label'].astype('string[pyarrow]')

In [57]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547557 entries, 0 to 547556
Data columns (total 91 columns):
 #   Column                      Non-Null Count   Dtype                 
---  ------                      --------------   -----                 
 0   id                          547557 non-null  int64[pyarrow]        
 1   Flow ID                     547557 non-null  string                
 2   Src IP                      547557 non-null  string                
 3   Src Port                    547557 non-null  int64[pyarrow]        
 4   Dst IP                      547557 non-null  string                
 5   Dst Port                    547557 non-null  int64[pyarrow]        
 6   Protocol                    547557 non-null  int64[pyarrow]        
 7   Timestamp                   547557 non-null  timestamp[ns][pyarrow]
 8   Flow Duration               547557 non-null  int64[pyarrow]        
 9   Total Fwd Packet            547557 non-null  int64[pyarrow]        
 10  Total Bw